In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from hydroeval import evaluator, nse, kge, pbias
from tqdm import tqdm
import datetime as dt

In [8]:
pnwNP = pd.read_csv("../data/pnwNP_StatsLocations_SS.csv")

colnm = ["time",'obs','model','water_year']
out_col = ['gage','model','nse','kge','kge_r','kge_a','kge_b','n_wateryear']
# df_list = ['vic','prms','nwm0','nwm1']
# df_list = [VIC,PRMS,NWM0,NWM1]

out_df = pd.DataFrame(columns = out_col)

i = 1
# for i in tqdm(range(0,len(pnwNP))):

    try:
        ## Read in streamflow and simulation dataset
        df = pd.read_csv('../data/pnwNP_modeledData/'+str(pnwNP['gage'][i])+".csv")
        df['time'] = pd.to_datetime(df['time'])
        df['water_year'] = df.time.dt.year.where(df.time.dt.month < 10, df.time.dt.year + 1)


        ## subset the dataset to only overlapping time periods
        VIC = df[(df['streamflow_NWIS'].notna())&(df['streamflow_VIC'].notna())][['time','streamflow_NWIS','streamflow_VIC','water_year']].reset_index(drop=True)
        VIC.columns = colnm
        VIC.name = "vic"

        PRMS = df[(df['streamflow_NWIS'].notna())&(df['streamflow_PRMS'].notna())][['time','streamflow_NWIS','streamflow_PRMS','water_year']].reset_index(drop=True)
        PRMS.columns = colnm
        PRMS.name = "prms"

        NWM0 = df[(df['streamflow_NWIS'].notna())&(df['streamflow_NWM2d0'].notna())][['time','streamflow_NWIS','streamflow_NWM2d0','water_year']].reset_index(drop=True)
        NWM0.columns = colnm
        NWM0.name = "nwm0"

        NWM1 = df[(df['streamflow_NWIS'].notna())&(df['streamflow_NWM2d1'].notna())][['time','streamflow_NWIS','streamflow_NWM2d1','water_year']].reset_index(drop=True)
        NWM1.columns = colnm
        NWM1.name = "nwm1"

        temp = NWM1

        df_list = [VIC,PRMS,NWM0,NWM1]

        for j, df in enumerate(df_list):
            temp = pd.DataFrame(df_list[j])
            ## Calculate hydrovariables
            NSE = evaluator(nse,
                            simulations = temp['model'],
                            evaluation = temp['obs'], 
                            axis=1,
                            transform='inv',
                            epsilon=0.00001)

            KGE = evaluator(kge,
                            simulations = temp['model'],
                            evaluation = temp['obs'], 
                            axis=1,
                            transform='inv',
                            epsilon=0.00001)

            gage = pnwNP['gage'][i]


            # ## Return hydrovaribles and write to list
            out = pd.DataFrame(np.column_stack([int(gage),df_list[j].name,NSE,KGE,len(np.unique(temp["water_year"]))]))
            out.columns = out_col

            out_df = pd.concat([out_df,out])

    except:
        print("No data")
        pass
    

IndentationError: unexpected indent (616950892.py, line 13)

In [14]:
dat = pd.read_csv("../data/pnwNP_StatsLocations.csv")
out_df["n_wateryear"] = out_df["n_wateryear"].astype(int)
tt = out_df
tt = tt.reset_index(drop=True)
tt['gage']=tt['gage'].astype(int)
tt = tt.merge(dat, how='inner', on='gage')

In [15]:
tt.to_csv("../data/pnwNP_modelStats_NewEps.csv")

In [16]:
tt

,gage,model,nse,kge,kge_r,kge_a,kge_b,n_wateryear,site_no,dec_lat_va,dec_long_va,numZero,numOne,startDate,endDate,comid,numObs,propZero,propOne
0,10361700,vic,nan,nan,nan,nan,nan,0,10361700,41.722116,-119.373258,2797,3228,1963-10-02,1972-09-29,24061841.0,3285,0.851446,0.982648
1,10361700,prms,nan,nan,nan,nan,nan,0,10361700,41.722116,-119.373258,2797,3228,1963-10-02,1972-09-29,24061841.0,3285,0.851446,0.982648
2,10361700,nwm0,nan,nan,nan,nan,nan,0,10361700,41.722116,-119.373258,2797,3228,1963-10-02,1972-09-29,24061841.0,3285,0.851446,0.982648
3,10361700,nwm1,nan,nan,nan,nan,nan,0,10361700,41.722116,-119.373258,2797,3228,1963-10-02,1972-09-29,24061841.0,3285,0.851446,0.982648
4,10366000,vic,nan,nan,nan,nan,nan,0,10366000,42.072111,-119.962725,0,61,1950-01-02,1991-10-17,24042897.0,15263,0.000000,0.003997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,480608115242901,prms,nan,nan,nan,nan,nan,0,480608115242901,48.102226,-115.408919,96,1305,2011-10-02,2020-12-31,22893835.0,3378,0.028419,0.386323
2014,480608115242901,nwm0,-239820.50364881472,-486.912465423354,0.24417036086411645,487.4035988582439,39.334601273532506,8,480608115242901,48.102226,-115.408919,96,1305,2011-10-02,2020-12-31,22893835.0,3378,0.028419,0.386323
2015,480608115242901,nwm1,-0.029389091279884427,-0.5370439376267249,0.38070280940713,0.0017572668075630114,0.008795429012083496,10,480608115242901,48.102226,-115.408919,96,1305,2011-10-02,2020-12-31,22893835.0,3378,0.028419,0.386323
2016,480608115242901,nwm0,-239820.50364881472,-486.912465423354,0.24417036086411645,487.4035988582439,39.334601273532506,8,480608115242901,48.102226,-115.408919,96,1305,2011-10-02,2020-12-31,22893835.0,3378,0.028419,0.386323
